In [ ]:
from elasticsearch import Elasticsearch, helpers
import requests
import json
import pandas as pd
import re
import ast
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import shape, MultiPolygon
import pyecharts.options as opts
from pyecharts.charts import Timeline, Bar, Grid, Pie, Line
from pyecharts.globals import ThemeType
import warnings
warnings.filterwarnings('ignore')
from requests.exceptions import RequestException
requests.packages.urllib3.disable_warnings()


import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns


In [ ]:
url = 'https://localhost:9200'
headers = {'Content-Type': 'application/json'}
auth = ('elastic', 'elastic')
requests.packages.urllib3.disable_warnings()
verify_ssl = False
client = Elasticsearch (
        'https://localhost:9200',
        verify_certs= False,
        ssl_show_warn= False,
        basic_auth=auth
    )

In [ ]:
search_body = {
    "query": {
        "match_all": {}
    }}
geo_response = client.search(index='geodata', body=search_body, size = 1000)

In [ ]:
hits = geo_response['hits']['hits']
data = [hit['_source'] for hit in hits]
df = pd.DataFrame(data)

## Geo Data Load

In [ ]:
def convert_to_shape(geo_dict):
    return shape(geo_dict)
df['geometry'] = df['geometry'].apply(convert_to_shape)
geo_data = gpd.GeoDataFrame(df, geometry='geometry')
geo_data = geo_data.rename(columns={
    'State_name_2021': 'state_name',
    'LGA_code_2023': 'lga_code',
    'LGA_name_2023': 'lga_name'
})

In [ ]:
crime = pd.read_csv('data/crime.csv')
economy = pd.read_csv('data/economy.csv')
population = pd.read_csv('data/population.csv')

population = population.fillna(0)
crime = crime.fillna(0)
economy = economy.fillna(0)

## Transform Data into Formal Dataframe

In [ ]:
homeless_url = 'http://localhost:9090/homeless'
homeless_response = requests.get(homeless_url)
if homeless_response.status_code == 200:
    json_data = homeless_response.json()
    homeless_df = pd.DataFrame(json_data)
else:
    print(f"Request failed with status code: {homeless_response.status_code}")

In [ ]:
homeless_state_df = pd.merge(geo_data, homeless_df, on = ['lga_code'], how = 'inner')
homeless_state_df['year'] = homeless_state_df['year'].astype(int)
# homeless_state_df = homeless_state_df.drop(columns = 'geometry')
homeless_state_df = homeless_state_df.drop(columns = 'lga_name_y')
homeless_state_df = homeless_state_df.rename(columns={
    'lga_name_x': 'lga_name'})

In [ ]:
state_total = homeless_state_df.groupby('state_name')['total'].sum().reset_index()
state_categories = homeless_state_df.groupby('state_name')[['at_risk', 'homeless', 'not_state']].sum().reset_index()

states = state_total['state_name'].tolist()
total_values = state_total['total'].tolist()
at_risk_values = state_categories['at_risk'].tolist()
homeless_values = state_categories['homeless'].tolist()
not_state_values = state_categories['not_state'].tolist()

bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(states)
    .add_yaxis("At Risk", at_risk_values, stack="stack1", category_gap="50%", label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("Homeless", homeless_values, stack="stack1", category_gap="50%", label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("Not State", not_state_values, stack="stack1", category_gap="50%", label_opts=opts.LabelOpts(is_show=False))
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="Total",
            type_="value",
            min_=0,
            max_=max(total_values) * 1.2,
            interval=max(total_values) / 5,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Total and Categories by State"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="shadow"),
        xaxis_opts=opts.AxisOpts(type_="category"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name="Counts",
            axislabel_opts=opts.LabelOpts(formatter="{value}", font_size=10),
        ),
    )
)

# Create a line chart
line = (
    Line()
    .add_xaxis(states)
    .add_yaxis(
        "Total", 
        total_values, 
        yaxis_index=1, 
        label_opts=opts.LabelOpts(is_show=False)
    )
)

# Combine bar and line charts
bar.overlap(line).render_notebook()

In [ ]:
def format_data(dataframe):
    years = dataframe['year'].unique()
    formatted_data = {}
    for year in years:
        yearly_data = dataframe[dataframe['year'] == year]
        top10_data = yearly_data.nlargest(10, 'total')
        formatted_data[year] = {
            'lga_name': top10_data['lga_name'].tolist(),
            'at_risk': top10_data['at_risk'].tolist(),
            'homeless': top10_data['homeless'].tolist(),
            'not_state': top10_data['not_state'].tolist()
        }
    return formatted_data
victoria_data = homeless_state_df[homeless_state_df['state_name'] == 'Victoria'].sort_values(by='year')
formatted_data = format_data(victoria_data)

In [ ]:
def get_year_overlap_chart(year):
    bar = (
        Bar()
        .add_xaxis(xaxis_data=formatted_data[year]['lga_name'])
        .add_yaxis(
            series_name="At Risk",
            y_axis=formatted_data[year]['at_risk'],
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="Homeless",
            y_axis=formatted_data[year]['homeless'],
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="Not State",
            y_axis=formatted_data[year]['not_state'],
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="{} Homelessness Data".format(year), subtitle="Data by LGA"
            ),
            tooltip_opts=opts.TooltipOpts(
                is_show=True, trigger="axis", axis_pointer_type="shadow"
            ),
            legend_opts=opts.LegendOpts(
                selected_map={
                    "At Risk": True,
                    "Homeless": True,
                    "Not State": True,
                }
            ),
            xaxis_opts=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(rotate=10, font_size=10)
            )
        )
    )
    pie = (
        Pie()
        .add(
            series_name="Homelessness Breakdown",
            data_pair=[
                ["At Risk", sum(formatted_data[year]['at_risk'])],
                ["Homeless", sum(formatted_data[year]['homeless'])],
                ["Not State", sum(formatted_data[year]['not_state'])],
            ],
            center=["75%", "35%"],
            radius="28%",
        )
        .set_series_opts(tooltip_opts=opts.TooltipOpts(is_show=True, trigger="item"))
    )
    return bar.overlap(pie)

timeline = Timeline(init_opts=opts.InitOpts(
    theme=ThemeType.LIGHT,
    width="1200px",
    height="800px"
))

for y in formatted_data.keys():
    timeline.add(get_year_overlap_chart(year=y), time_point=str(y))

timeline.add_schema(is_auto_play=False, play_interval=3000)
timeline.render_notebook()

### Economy Data Transform

In [ ]:
transformed_economy_data = []

for column in economy.columns:
    if column not in ['fin_yr', ' lga_name_2019', ' lga_code_2019'] and all(x not in column for x in ['bsnss_entrs','bsnss_entrs','indstry_emplynt', 'nmbr_bsnsss', 'registered_motor_vehicles', 'rgstrd_mtr_vhcls']):
        if 'business_entries' in column:
            type_ = 'business_entries'
        elif 'business_exits' in column:
            type_ = 'business_exits'
        elif 'number_of_businesses' in column and 'construction' in column:
            type_ = 'construction_businesses'
        elif 'number_of_businesses' in column and 'mining' in column:
            type_ = 'mining_businesses'
        elif 'number_of_businesses' in column and 'retail_trade' in column:
            type_ = 'retail_trade_businesses'
        elif 'ocptns_dbtrs_entrg' in column and 'clr_ad' in column:
            type_ = 'clerical_administrative_debtors'
        elif 'ocptns_dbtrs_entrg' in column and 'cmnty' in column:
            type_ = 'community_personal_service_debtors'
        elif 'ocptns_dbtrs_entrg' in column and 'lbrs_n' in column:
            type_ = 'labour_debtors'
        elif 'ocptns_dbtrs_entrg' in column and 'mchnry' in column:
            type_ = 'machinery_operators_and_drivers_debtors'
        elif 'ocptns_dbtrs_entrg' in column and 'mngrs' in column:
            type_ = 'manager_debtors'
        elif 'ocptns_dbtrs_entrg' in column and 'or_unk' in column:
            type_ = 'debtors_with_unknown_occupations'
        elif 'ocptns_dbtrs_entrg' in column and 'prfsnl' in column:
            type_ = 'professional_debtors'
        elif 'ocptns_dbtrs_entrg' in column and 'sl_wrk' in column:
            type_ = 'sales_debtors'
        elif 'ocptns_dbtrs_entrg' in column and 'tchns' in column:
            type_ = 'technicians_and_trades_debtors'
        elif 'prsnl_inslvncs' in column:
            type_ = 'personal_insolvencies'
        elif 'attchd_dwllngs_mdn_sle_prce' in column:
            type_ = 'median_price_attached_dwellings'
        elif 'hss_mdn_sle_prce' in column:
            type_ = 'median_sale_price_houses'
        elif 'hss_nmbr_trnsfrs_nm' in column:
            type_ = 'transfer_houses_num'
        elif 'atchd_dwlngs_nmbr_trnsf_nm' in column:
            type_ = 'transfer_attached_dwellings_num'
        elif 'bsn_rltd_csd_ecnm_cndts_nm' in column:
            type_ = 'personal_insolvencies_economic'
        elif 'bsn_rltd_csd_excsv_use_of' in column:
            type_ = 'personal_insolvencies_excessive_credit'
        elif 'bsn_rltd_csd_unmplyt_ls_o' in column:
            type_ = 'personal_insolvencies_loss_of_income'

        # 遍历每一行
        for index, value in economy[column].items():
            row_data = {
                'year': economy[' yr'][index],
                'lga_name': economy[' lga_name_2019'][index],
                'lga_code': economy[' lga_code_2019'][index],
                'type': type_,
                'count': value
            }
            transformed_economy_data.append(row_data)

transformed_economy_df = pd.DataFrame(transformed_economy_data)

In [ ]:
pivot_econmy_df = transformed_economy_df.pivot_table(index=['year', 'lga_name', 'lga_code'], columns='type', values='count', aggfunc = 'sum', fill_value=0)
pivot_econmy_df.reset_index(inplace=True)

In [ ]:
pivot_econmy_df['debtor_num'] = pivot_econmy_df[[col for col in pivot_econmy_df.columns if 'debtors' in col]].sum(axis=1)
pivot_econmy_df['businesses_num'] = pivot_econmy_df[[col for col in pivot_econmy_df.columns if 'businesses' in col]].sum(axis=1)
pivot_econmy_df['personal_insolvencies_num'] = pivot_econmy_df[[col for col in pivot_econmy_df.columns if 'personal_insolvencies' in col]].sum(axis=1)
cols_to_keep = [col for col in pivot_econmy_df.columns if not any(x in col for x in ['debtors', 'businesses', 'personal_insolvencies'])]
cols_to_keep.extend(['businesses_num', 'personal_insolvencies_num'])
economy_filtered = pivot_econmy_df[cols_to_keep]

In [ ]:
economy_filtered